## Import

In [ ]:
import numpy as np
import random
import tensorflow as tf
from keras.layers import Dense,LSTM,Bidirectional,Input,Concatenate,Embedding,Attention,TimeDistributed
from keras.models import Model
from keras.callbacks import EarlyStopping
import math
import pandas as pd
from scipy.stats import pearsonr,spearmanr
nan = np.nan

## Initialisations

In [ ]:
# TYPE/TOKEN FREQUENCY
use_token_freq = False # True for token frequency model, false for type frequency model
square_root_normalization = False # True for token frequency model, false for type frequency model

# HYPERPARAMETER SETTINGS
latent_dim = 100
embedding_dim = 300
batch_size = 32
patience = 15
dropout = 0.3
lr = 0.001

# FIXED SETTINGS after Experiment 1
track = 'val_loss' #'val_loss' for validation loss / 'loss' for training loss tracking with early stopping criterion
excl_extra_verbs = False
remove_doubles = False
SEED = 4 # Same seed for same data split across different expeirments
val_split = 0.8 # Do not change, because we use 5 different data splits based on an 80/20 split
beam_width = 12
max_epochs = 400 # Early stopping is used so max is never reached

In [ ]:
# Choose one: experiment 1 / experiment 2 / both
exp1 = True
exp2 = False
exp_1_2 = False

# Run only default data fold 5 times (as done for experiment 1) -- referred to as split 1 in the report
if exp1:
    number_of_initialisations = 5
    fold = 'DEFAULT'

# Run the 4 other data splits 5 times, in addition to the one of experiment 1 (as done for experiment 2)
if exp2:
    number_of_initialisations = 20
    fold = 'first20'

# Run all data splits 5 times
if exp_1_2:
    number_of_initialisations = 25
    fold = 'first20'

# Functions needed during the experiment

Note that:
- "split 1" in the thesis report refers to the "DEFAULT" split in this code! This datasplit used for Experiment 1.
- "val"/"validation" here means "development set" that is mentioned in the thesis report.

## Load and split data (depending on experiment settings)

In [ ]:
def all_data(square_root_normalization, SEED, val_split, fold, excl_extra_verbs, remove_doubles):
    # CELEX verb set based on past tense form frequency => 1
    with open("my_dataset.txt", encoding="utf8") as dataset:
        verblist = [line.strip().split("\t") for line in dataset][1:]
    # A&H data set from K&C's repository, CELEX verb set based on lemma frequency > 10
    with open("KCamerican.txt", encoding="utf8") as KC: #
        KClist = [line.strip().split("\t") for line in KC][1:]
    
    # Initialize lists
    freq, pairs, written_pairs, label = [], [], [], []
    
    # Find intersection of datasets based on specific columns
    for lemma in verblist:
        for KClemma in KClist:
            if KClemma[3] == lemma[2] and KClemma[4] == lemma[3]:
                pairs.append([KClemma[0], KClemma[1]])          # IPA verb pairs (present/past tense) from K&C data
                written_pairs.append([lemma[2], lemma[3]])      # Verb pairs in written form
                freq.append(int(lemma[0]))                      # Past tense token frequency
                label.append(KClemma[-1])                       # Morphological class (regular/irregular) from K&C data
    
    # Add missing verbs from an external file (verbs that appeared to be omitted from A&H's data)
    with open("missing.txt", encoding="utf8") as missing:
        additional_verbs = [line.strip().split("\t") for line in missing]
        
    for verb in additional_verbs:
        pairs.insert(0, [verb[2], verb[3]])                    # Insert at the beginning
        written_pairs.insert(0, [verb[0], verb[1]])
        freq.insert(0, int(verb[4]))
        label.insert(0, verb[5])
    
    # consolidate identical examples
    new_pairs, new_written_pairs, new_freq, new_label = [], [], [], []
    
    for i, wp in enumerate(written_pairs):
        if wp in new_written_pairs:
            idx = new_written_pairs.index(wp)
            new_freq[idx] += freq[i]  # Sum token frequency for duplicates
        else:
            new_written_pairs.append(wp)
            new_pairs.append(pairs[i])
            new_freq.append(freq[i])
            new_label.append(label[i])
    
    # Update variables with cleaned data
    written_pairs, pairs, freq, label = new_written_pairs, new_pairs, new_freq, new_label
    
    # Normalize token frequencies using square root if needed
    if square_root_normalization:
        freq = [int(math.sqrt(f)) for f in freq]
        
    # Shuffle data with a fixed seed
    for lst in [pairs, written_pairs, freq, label]:
        random.Random(SEED).shuffle(lst)

    # Split data into train-validation sets
    split_idx = int(val_split * len(pairs))
    train_pairs, val_pairs = pairs[:split_idx], pairs[split_idx:]
    train_wp, val_wp = written_pairs[:split_idx], written_pairs[split_idx:]
    train_freq, val_freq = freq[:split_idx], freq[split_idx:]
    train_label, val_label = label[:split_idx], label[split_idx:]

    # Calculate fold boundaries for splitting the data into five folds (80/20 train-val)
    fold_size = len(val_pairs)
    fold2, fold3, fold4, fold5 = fold_size, fold_size * 2, fold_size * 3, fold_size * 4

    # Create train-validation splits based on the fold selected. If fold == 'DEFAULT' (Experiment 1) then data is split as fold 5 / fifth 20%
    if fold == 'first20':
        val_pairs, train_pairs = pairs[:fold2], pairs[fold2:]
        val_wp, train_wp = written_pairs[:fold2], written_pairs[fold2:]
        val_freq, train_freq = freq[:fold2], freq[fold2:]
        val_label, train_label = label[:fold2], label[fold2:]
        
    elif fold == 'second20':
        val_pairs, train_pairs = pairs[fold2:fold3], pairs[:fold2] + pairs[fold3:]
        val_wp, train_wp = written_pairs[fold2:fold3], written_pairs[:fold2] + written_pairs[fold3:]
        val_freq, train_freq = freq[fold2:fold3], freq[:fold2] + freq[fold3:]
        val_label, train_label = label[fold2:fold3], label[:fold2] + label[fold3:]
        
    elif fold == 'third20':
        val_pairs, train_pairs = pairs[fold3:fold4], pairs[:fold3] + pairs[fold4:]
        val_wp, train_wp = written_pairs[fold3:fold4], written_pairs[:fold3] + written_pairs[fold4:]
        val_freq, train_freq = freq[fold3:fold4], freq[:fold3] + freq[fold4:]
        val_label, train_label = label[fold3:fold4], label[:fold3] + label[fold4:]
        
    elif fold == 'fourth20':
        val_pairs, train_pairs = pairs[fold4:fold5], pairs[:fold4] + pairs[fold5:]
        val_wp, train_wp = written_pairs[fold4:fold5], written_pairs[:fold4] + written_pairs[fold5:]
        val_freq, train_freq = freq[fold4:fold5], freq[:fold4] + freq[fold5:]
        val_label, train_label = label[fold4:fold5], label[:fold4] + label[fold5:]
    
    # Experiment 1: 
    # If desired, remove the added missing verbs
    if excl_extra_verbs:
        removelist = additional_verbs   # Remove the verbs that were added (seems redundant, but they should not be excluded before shuffling and splitting the data as that would lead to different splits compared to the configuration where we keep these verbs in the data)
        for p in removelist:
            if p in train_pairs:
                n = train_pairs.index(p)
                train_pairs.pop(n)
                train_wp.pop(n)
                train_label.pop(n)
                train_freq.pop(n)
            elif p in val_pairs:
                n = val_pairs.index(p)
                val_pairs.pop(n)
                val_wp.pop(n)
                val_label.pop(n)
                val_freq.pop(n)
    
    # Experiment 1:                
    # If desired, remove the second(+) inflections of verbs that have more than one correct inflection
    if remove_doubles:
        removelist,seen,indices=[],[],[]
        for i,p in enumerate(pairs):
            if p[0] in seen:
                for j,s in enumerate(seen):
                    if p[0]==s and p[1]!=pairs[indices[j]][1]:  # present==present and past!=past (can also be with past==past)
                        frequency = freq[i]
                        prev_seen_freq = freq[indices[j]]
                        if frequency >= prev_seen_freq:     # Keep the most frequent example
                            removelist.append(pairs[indices[j]])
                        else:
                            removelist.append(p)
            seen.append(p[0])
            indices.append(i)
        for p in removelist:
            if p in train_pairs:
                n = train_pairs.index(p)
                train_pairs.pop(n)
                train_wp.pop(n)
                train_label.pop(n)
                train_freq.pop(n)
            elif p in val_pairs:
                n = val_pairs.index(p)
                val_pairs.pop(n)
                val_wp.pop(n)
                val_label.pop(n)
                val_freq.pop(n)
    
    # Concatenate training and validation sets again
    pairs = train_pairs + val_pairs
    written_pairs = train_wp + val_wp
    label = train_label + val_label
    freq = train_freq + val_freq
    split = len(train_pairs)
    
    return pairs, written_pairs, freq, label, split

## pre-process data to encoder-decoder inputs/targets

In [ ]:
def create_model_data(split, pairs, freq, use_token_freq):
    # Create input data and target data for the encoder-decoder model
    verbs, input_verbs, target_verbs = [],[],[]
    new_pairs = []
    characters = set() 
    
    for input_verb, target_verb in pairs: 
        iv,tv = "",""                               
        
        # Process input verbs (present tense forms)
        for X,char in enumerate(input_verb):
            # Fixing phonetic transcription from the K&C data
            if char == '\x8d': 
                char = 'A'
            iv += char
            # Add every character to the character set, in order to create character_indexation later on
            characters.add(char)
        input_verbs.append(iv)
        
        # The same for the target verbs (past tense forms) and add start sign '\t' and end sign '\n'
        target_verb = "\t" + target_verb + "\n"   
        for Y,char in enumerate(target_verb):
            if char == '\x8d':
                char = 'A'
            tv += char
            characters.add(char)
        new_pairs.append([iv, tv.strip()]) # new_pairs to update to 'pairs' with the fixed 'A' character
        target_verbs.append(tv) 
        
    pairs = new_pairs # new_pairs to update to 'pairs' with the fixed 'A' character
    
    # Create a character index
    characters = sorted(list(characters))
    char_index = dict([(char, i) for i, char in enumerate(characters)])
    
    # Number of characters
    num_characters = len(characters)

    # Compute max word length and number of samples (verb pairs) to determine array sizes below
    verbs = input_verbs + target_verbs
    max_word_length = max([len(verb) for verb in verbs])
    num_samples = len(pairs)
    
    # Initialize encoder and decoder input/target data
    encoder_input_data = np.zeros((num_samples, max_word_length), dtype="float32") # Representing present tense forms
    decoder_input_data = np.zeros((num_samples, max_word_length), dtype="float32") # Representing correct past tense forms for teacher forcing during training
    decoder_target_data = np.zeros((num_samples, max_word_length), dtype="float32") # Representing correct past tense forms
    
    # Fill encoder and decoder data
    for i, (input_verb, target_verb) in enumerate(zip(input_verbs, target_verbs)):
        for j, char in enumerate(input_verb):
            encoder_input_data[i, j] =  char_index[char]
        for j, char in enumerate(target_verb):
            decoder_input_data[i, j] =  char_index[char]
            if j > 0:
                decoder_target_data[i, j-1] = char_index[char]  # Exclude start character \t", the target sequence is one character ahead of the input (i.e. the previous correct character) 

   # Training encoder input data (eid), decoder input data (did), decoder target data (dtd)
    train_eid = encoder_input_data[:split]
    train_did = decoder_input_data[:split]
    train_dtd = decoder_target_data[:split]
    train_target_verbs = target_verbs[:split]

    # Validation encoder input data (eid), decoder input data (did), decoder target data (dtd)
    val_eid = encoder_input_data[split:]
    val_did = decoder_input_data[split:]
    val_dtd = decoder_target_data[split:]
    val_target_verbs = target_verbs[split:]
    val_data = ([val_eid, val_did], val_dtd)

    # If desired (Experiment 1 + 2) augment training part with token frequency
    if use_token_freq:
        num_samples=0
        
        # First compute total number of examples if using token frequency
        for n,__ in enumerate(train_eid):
            for f in range(freq[n]):
                num_samples+=1  
                
        # Initialise new enc/dec input/target data with the updated total number of examples
        encoder_input_data = np.zeros((num_samples, max_word_length), dtype="float32")
        decoder_input_data = np.zeros((num_samples, max_word_length), dtype="float32")
        decoder_target_data = np.zeros((num_samples, max_word_length), dtype="float32")
                
        # Augment training encoder and decoder input/target data by adding N examples, where N is the token frequency of the verb
        t=0
        for n,x in enumerate(train_eid):
            for f in range(freq[n]):
                encoder_input_data[t] = train_eid[n]
                decoder_input_data[t] = train_did[n]
                decoder_target_data[t] = train_dtd[n]
                t+=1  
        
    else:
        encoder_input_data=train_eid
        decoder_input_data=train_did
        decoder_target_data=train_dtd

    return encoder_input_data, train_eid, train_target_verbs, decoder_input_data, decoder_target_data, val_eid, val_target_verbs, char_index, max_word_length, num_characters, val_data

## Load Wug data function

In [ ]:
def all_wug_data(char_index,max_word_length):
    wugs,wugs_reg,wugs_irreg1,wugs_irreg2=[],[],[],[]
    
    # Load all wug forms
    with open("KCamericanwugs.txt", encoding="utf8") as KCwug_data: # A&H nonce verb present tense in K&C AE phonetic transcriptions
        for wug in KCwug_data:
            wugs.append("".join(wug.split()))
            
    with open("reg.txt", encoding="utf8") as KCwug_data: # A&H regular inflection in K&C AE phonetic transcriptions
        for wug in KCwug_data:
            wugs_reg.append("".join(wug.split()))       
            
    with open("irreg1.txt", encoding="utf8") as KCwug_data: # A&H irregular inflection in K&C AE phonetic transcriptions
        for wug in KCwug_data:
            wugs_irreg1.append("".join(wug.split()))    
            
    with open("irreg2.txt", encoding="utf8") as KCwug_data: # A&H 2nd option irregular inflection in K&C AE phonetic transcriptions
        for wug in KCwug_data:
            wugs_irreg2.append("".join(wug.split()))
    
    # Convert the wug verbs into arrays of integers in the same way as the real verb data
    wug_input_data = np.zeros((len(wugs), max_word_length), dtype="float32")
    
    for i, input_nonce in enumerate(wugs):
        for t, char in enumerate(input_nonce):
            wug_input_data[i, t] =  char_index[char]
            
    return wugs, wugs_reg ,wugs_irreg1, wugs_irreg2, wug_input_data

## Function to train and build Encoder-Decoder model

In [ ]:

def train_and_build_model(num_characters,max_word_length, encoder_input_data, decoder_input_data, decoder_target_data, val_data):
    
    ## ENCODER ##
    encoder_inputs = Input(shape=(max_word_length,))
        
    # Embedding layer
    enc_emb_layer = Embedding(num_characters, embedding_dim, trainable=True)
    enc_emb = enc_emb_layer(encoder_inputs)

    # BiLSTM layer 1
    enc_lstm1 = Bidirectional(LSTM(int(latent_dim/2), return_sequences=True, return_state=True, dropout=dropout))
    encoder_output1, f_h, f_c, b_h, b_c = enc_lstm1(enc_emb, training=True)

    c1, c2 = Concatenate(), Concatenate()
    e_h = c1([f_h, b_h])
    e_c = c2([f_c, b_c])
    encoder_states1 = [e_h, e_c]
        
    # BiLSTM layer 2
    enc_lstm2 = Bidirectional(LSTM(int(latent_dim/2), return_sequences=True, return_state=True, dropout=dropout))
    encoder_output2, f_h2, f_c2, b_h2, b_c2 = enc_lstm2(encoder_output1, training=True)

    c3, c4 = Concatenate(), Concatenate()
    e_h2 = c3([f_h2, b_h2])
    e_c2 = c4([f_c2, b_c2])
    encoder_states2 = [e_h2, e_c2]

    ## DECODER ##
    decoder_inputs = Input(shape=(None,))
    
    # Embedding layer
    dec_emb_layer = Embedding(num_characters, embedding_dim, trainable=True)
    dec_emb = dec_emb_layer(decoder_inputs)

    # LSTM layer 1
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=dropout)
    dec_out, d_h, d_c = decoder_lstm(dec_emb, initial_state=encoder_states1, training=True)
    decoder_states = [d_h, d_c]
    
    # LSTM layer 2
    decoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=dropout)
    dec_out2, d_h2, d_c2 = decoder_lstm2(dec_out, initial_state=encoder_states2, training=True)
    decoder_states2 = [d_h2, d_c2]

    ## ATTENTION ##
    ED_attention = Attention()
    ed_attention = ED_attention([dec_out2, encoder_output2])

    decoder_concat_layer = Concatenate(axis=-1)
    dense_input = decoder_concat_layer([dec_out2, ed_attention])

    ## DENSE ##
    decoder_dense = TimeDistributed(Dense(num_characters, activation='softmax')) 
    decoder_outputs = decoder_dense(dense_input)
    
    ## COMPILE AND FIT MODEL ##
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)  
    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor=track, patience=patience, restore_best_weights=True)

    history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data, epochs=max_epochs,
                        batch_size=batch_size, validation_data=val_data, callbacks=[early_stopping], shuffle=True)
    
    print(len(history.history['loss']))
    
    # Return epoch number to which weights are restored (best ones)
    n_epochs = len(history.history['loss'])-patience

    ## BUILD INFERENCE MODEL ##
    
    # Encoder
    enc_emb = enc_emb_layer(encoder_inputs) # Embedding
    encoder_output1, f_h, f_c, b_h, b_c = enc_lstm1(enc_emb, training=False)
    e_h = c1([f_h, b_h])
    e_c = c2([f_c, b_c])
    encoder_states1 = [e_h, e_c] # BiLSTM layer 1
    encoder_output2, f_h2, f_c2, b_h2, b_c2 = enc_lstm2(encoder_output1, training=False)
    e_h2 = c3([f_h2, b_h2])
    e_c2 = c4([f_c2, b_c2])
    encoder_states2 = [e_h2, e_c2] # BiLSTM layer 2

    encoder_model = Model(encoder_inputs, [encoder_output2] + encoder_states1 + encoder_states2)

    # Decoder
    e_states1 = [Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
    e_states2 = [Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
    e_output = Input(shape=(max_word_length, latent_dim))
    
    dec_emb = dec_emb_layer(decoder_inputs) # Embedding
    dec_out, d_h, d_c = decoder_lstm(dec_emb, initial_state=e_states1, training=False)
    decoder_states = [d_h, d_c] # LSTM layer 1 
    dec_out2, d_h2, d_c2 = decoder_lstm2(dec_out, initial_state=e_states2, training=False)
    decoder_states2 = [d_h2, d_c2] # LSTM layer 2
    
    ed_attention = ED_attention([dec_out2, e_output]) # Attention
    dense_input = decoder_concat_layer([dec_out2, ed_attention])
    decoder_outputs = decoder_dense(dense_input) # Main task

    decoder_model = Model([decoder_inputs] + [e_output] + e_states1 + e_states2,
                          [decoder_outputs] + decoder_states + decoder_states2)

    return encoder_model, decoder_model, n_epochs

## Function to decode sequences from ED output to verbs (character by character)

In [ ]:
def decode_sequence(input_seq, encoder_model, decoder_model, max_len, char_index):
    # Create a reverse character index, to convert predictions to characters
    reverse_char_index = dict((i, char) for char, i in char_index.items())

    # Encode input sequence (present tense verb form)
    encoder_output, h1, c1, h2, c2 = encoder_model.predict(input_seq, verbose=0)
    states_value1 = [h1, c1]
    states_value2 = [h2, c2]

    # Prepare target sequence (predicted past tense verb form)
    current_char = np.array([[char_index["\t"]]])   # Sequence starts with start sign '\t'
    stop_condition = False
    decoded_verb = []

    # Loop predicting the next character based on the previous prediction
    while not stop_condition:
        # Decoder prediction
        output, h1, c1, h2, c2 = decoder_model.predict([current_char] + [encoder_output] + states_value1 + states_value2, verbose=0)
        
        # Get index of predicted character
        predicted_char_index = np.argmax(output[0, 0])
        predicted_char = reverse_char_index[predicted_char_index]
        
        # Append predicted character
        decoded_verb.append(predicted_char)

        # Stopping condition is true if case length exceeds max word length or stop sign '\n' is found
        if predicted_char == "\n" or len(decoded_verb) > max_len:
            stop_condition = True
        else:
            # Updating states to predicted character
            current_char[0, 0] = predicted_char_index
            states_value1 = [h1, c1]
            states_value2 = [h2, c2]

    # Return the decoded verb
    return ''.join(decoded_verb)


## Beam Search algorithm (used in the Wug Test)

In [ ]:
def beam_search(input_seq, enc_model, dec_model, beam_width, max_len, char_index):
    
    # Beam search algorithm used for nonce verb (Wug Test) predictions

    # Predict encoding of input sequence (present tense verb)
    encoded_seq, h1, c1, h2, c2 = enc_model.predict(input_seq,verbose=0)
    state1 = [h1, c1] # Encoder BiLSTM layer 1 states
    state2 = [h2, c2] # Encoder BiLSTM layer 2 states
    
    current_char = np.zeros((1, 1))
    current_char[0, 0] = char_index["\t"] # Sequence always start with start sign \t, so current character at the beginning is \t
    total_seq = current_char
    
    # Initialise beam
    beam = [{'current_char': current_char, 'prob': 1.0, 'state1': state1, 
             'state2':state2,'total_seq':total_seq}]
    
    # Compute the 12-most likely output sequences (predictions for the past tense form)
    for i in range(max_len):
        candidates = []  # List to hold all candidate sequences for this step
        
        # Loop over each sequence in the current beam
        for b in beam:
            current_char = b['current_char']   # Get the current character (previously predicted one)
            state1 = b['state1']  # Get the current state from the first encoder layer
            state2 = b['state2']  # Get the current state from the second encoder layer
            total_seq = b['total_seq']  # Get the total sequence so far
            
            # Predict the next character probabilities using the decoder model
            probs, h1, c1, h2, c2 = dec_model.predict([current_char] + [encoded_seq] + state1 + state2, verbose=0)
            
            probs = probs[0][0]  # Get the probabilities for the next character
            top_k = tf.argsort([probs], direction='DESCENDING')[0][:beam_width]  # Get the indices of the top-k probabilities
            
            # Generate new candidate sequences for the top-k characters
            for k in top_k:
                predicted_char = tf.expand_dims([k], 1)  # Create the predicted char tensor
                total_seq = tf.concat([total_seq, predicted_char], axis=1)  # Append the predicted character to the total sequence
                current_char = predicted_char  # Update current character
                
                candidate_prob = probs[k] * b['prob']  # Combine probabilities for total candidate probability
                
                # Update the states for the candidate sequence
                candidate_state1 = [h1, c1]
                candidate_state2 = [h2, c2]
                
                # Store the candidate sequence and its properties
                candidates.append({'current_char': current_char, 'prob': candidate_prob, 'state1': candidate_state1, 
                                   'state2': candidate_state2, 'total_seq': total_seq})
        
        # Sort the candidates by probability
        candidates.sort(key=lambda x: x['prob'], reverse=True)  
        # Retain only the top K sequences for the next iteration
        beam = candidates[:beam_width]  
    
    return beam


## Function to compute the real verb accuracies and save errors

In [ ]:
def compute_acc(label, train_eid, decode_sequence, enc_model, dec_model, char_index, train_target_verbs, val_target_verbs, pairs, val_eid, max_len):
    
    # Dictionary to hold more than 1 correct answer for inflections (given that in Exp 1 and 2 'doubles' are included)
    correct_answers = dict()
    seen = []
    
    # Combine training and validation sets to find 'doubles'
    all_eid = np.concatenate((train_eid, val_eid), axis=0)
    all_target_verbs = train_target_verbs + val_target_verbs

    # Fill correct_answers dictionary
    for i, t in enumerate(all_eid):
        t = all_eid[i:i + 1]
        if pairs[i][0] in seen:
            value = correct_answers[pairs[i][0]]
            new_value = value + [all_target_verbs[i].strip()]
            correct_answers[pairs[i][0]] = new_value
        else:
            seen.append(pairs[i][0])
            answer = all_target_verbs[i].strip()
            correct_answers[pairs[i][0]] = [answer]          
    
    # Compute training accuracy
    print("computing training accuracy")

    correct_ones, errors = [], []
    correct_train_regulars, correct_train_irregulars, error_train_regulars, error_train_irregulars = [], [], [], []

    for i, inputs in enumerate(train_eid):
        input_seq = train_eid[i:i + 1]
        decoded_verb = decode_sequence(input_seq, enc_model, dec_model, max_len, char_index)

        # Check if decoded verb is correct and save errors for analysis
        if decoded_verb.strip() in correct_answers[pairs[i][0]]:
            correct_ones.append([(pairs[i][0], pairs[i][1]), (decoded_verb.strip(), correct_answers[pairs[i][0]])]) # Present and past, decoded verb and correct answer
            if label[i] == 'reg':
                correct_train_regulars.append(pairs[i][1])
            else:
                correct_train_irregulars.append(pairs[i][1])
        else:
            errors.append([(pairs[i][0], pairs[i][1]), (decoded_verb.strip(), correct_answers[pairs[i][0]])]) # Present and past, decoded verb and correct answer
            if label[i] == 'reg':
                error_train_regulars.append([pairs[i][0], decoded_verb.strip(), correct_answers[pairs[i][0]]]) # Present, decoded verb and correct answer
            else:
                error_train_irregulars.append([pairs[i][0], decoded_verb.strip(), correct_answers[pairs[i][0]]]) # Present, decoded verb and correct answer

    # Calculate training accuracies for overall, regular, and irregular verbs
    train_accuracy = 100 * len(correct_ones) / (len(correct_ones) + len(errors))
    irreg_train_acc = 100 * len(correct_train_irregulars) / (len(correct_train_irregulars) + len(error_train_irregulars))
    reg_train_acc = 100 * len(correct_train_regulars) / (len(correct_train_regulars) + len(error_train_regulars))

    # Compute validation accuracy
    print("computing validation accuracy")

    val_correct_ones, val_errors = [], []
    correct_val_regulars, correct_val_irregulars, error_val_regulars, error_val_irregulars = [], [], [], []

    for i, inputs in enumerate(val_eid):
        j = i + len(train_eid)  # index of the item in the correct_answer dictionary and 'pairs' and 'label' lists (i-th example of the validation set + length of training set)
        input_seq = val_eid[i:i + 1]
        decoded_verb = decode_sequence(input_seq, enc_model, dec_model, max_len, char_index)

        # Check if decoded verb is correct and save both correct and incorrect inflections for analysis
        if decoded_verb.strip() in correct_answers[pairs[j][0]]:
            val_correct_ones.append([(pairs[j][0], pairs[j][1]), (decoded_verb.strip(), correct_answers[pairs[j][0]])]) # Present and past, decoded verb and correct answer
            if label[j] == 'reg':
                correct_val_regulars.append(pairs[j][1])
            else:
                correct_val_irregulars.append(pairs[j][1])
        else:
            val_errors.append([(pairs[j][0], pairs[j][1]), (decoded_verb.strip(), correct_answers[pairs[j][0]])]) # Present and past, decoded verb and correct answer
            if label[j] == 'reg':
                error_val_regulars.append([pairs[j][0], decoded_verb.strip(), correct_answers[pairs[j][0]]]) # Present, decoded verb and correct answer
            else:
                error_val_irregulars.append([pairs[j][0], decoded_verb.strip(), correct_answers[pairs[j][0]]])  # Present, decoded verb and correct answer

    # Calculate validation accuracies for overall, regular, and irregular verbs
    val_accuracy = 100 * len(val_correct_ones) / (len(val_correct_ones) + len(val_errors))
    irreg_val_acc = 100 * len(correct_val_irregulars) / (len(correct_val_irregulars) + len(error_val_irregulars))
    reg_val_acc = 100 * len(correct_val_regulars) / (len(correct_val_regulars) + len(error_val_regulars))   
    
    # Return all the computed accuracy and error details
    train_a_i_r = [train_accuracy, irreg_train_acc, reg_train_acc]
    val_a_i_r = [val_accuracy, irreg_val_acc, reg_val_acc]
    
    return train_a_i_r, val_a_i_r, error_train_regulars, error_train_irregulars, error_val_regulars, error_val_irregulars, correct_val_irregulars

## Compute and print correlation function

In [ ]:
def print_correlations(results):
    
    BEAM_PROD_CORR,BEAM_RATING_CORR = [],[]
    
    # A&H experiment results (production experiment and rating experiment)
    AH_df = pd.read_excel("AHresults.xlsx")
    AH_df_ratings = pd.read_excel("AHresults2.xlsx")

    # Structure model beam results in a similar way
    beam_results = [[r[0][1],r[1][1]] for r in results]
    beam_results_df = pd.DataFrame(beam_results, index=wugs, columns=['reg', 'irreg1'])
    
    # Regular wug inflection
    x=beam_results_df["reg"]
    y=AH_df["reg"]
    # Irregular wug inflection
    a=beam_results_df["irreg1"]
    b=AH_df["irreg1"] 

    # Correlation betweem BEAM and production/rating probabilities A&H
    print("Production probabilities vs. Beam probabilities")
    
    # Compute and print correlations beam probs vs A&H production prob data, regular class
    print("Regular class")
    corr, p_val = spearmanr(x, y)
    BEAM_PROD_CORR.append([corr, p_val])
    print("SPEARMAN:")
    print(f"r = {corr:.4f} (p = {p_val:.4f})")
    corr, p_val = pearsonr(x, y)
    BEAM_PROD_CORR.append([corr, p_val])
    print("PEARSON:")
    print(f"r = {corr:.4f} (p = {p_val:.4f})")

    # Compute and print correlations beam probs vs A&H production prob data, irregular class
    corr, p_val = spearmanr(a, b)
    BEAM_PROD_CORR.append([corr, p_val])
    corr, p_val = pearsonr(a, b)
    BEAM_PROD_CORR.append([corr, p_val])

    # Beam probs vs A&H rating data
    y2 = AH_df_ratings["reg"]
    b2 = AH_df_ratings["irreg1"]
    corr, p_val = spearmanr(x, y2)
    BEAM_RATING_CORR.append([corr, p_val])
    corr, p_val = pearsonr(x, y2)
    BEAM_RATING_CORR.append([corr, p_val])
    corr, p_val = spearmanr(a, b2)
    BEAM_RATING_CORR.append([corr, p_val])
    corr, p_val = pearsonr(a, b2)
    BEAM_RATING_CORR.append([corr, p_val])

    
    # In the analysis notebook this function is extended to print and compute the correlations between
        #the model's top 1 prediction as well. However, here, for individual runs of the model, it is not insightful.
    
    return BEAM_PROD_CORR,BEAM_RATING_CORR

# RUN EXPERIMENT ####


In [ ]:

# Load real and nonce verb data
pairs, written_pairs, freq, label, split = all_data(square_root_normalization, SEED, val_split, fold, excl_extra_verbs, remove_doubles)
encoder_input_data, train_eid, train_target_verbs, decoder_input_data, decoder_target_data, val_eid, val_target_verbs, char_index, max_word_length, num_characters, val_data = create_model_data(split, pairs, freq, use_token_freq)
wugs, wugs_reg, wugs_irreg1, wugs_irreg2, wug_input_data = all_wug_data(char_index, max_word_length)

# Create the reversed character index for decoding
reverse_char_index = dict((m, char) for char, m in char_index.items())

# Create the 'other' category for the Wug Test
wugs_other = [""] * len(wugs)

# Create list to save results per individual run/model
individual_results = [["model nr","fold", "num epochs", "train acc (All/Irreg/Reg)", "val_acc (All/Irreg/Reg)",
                       "correlations", "wug beam results", "top 1 forms", "beam per wug" ,                   
                      "reg train err", "irreg train err", "reg val err","irreg val err", "irreg val CORRECT"]]

## EXPERIMENT ##

for n in range(number_of_initialisations):
    model_nr = n+1
    print("Training model number: ", model_nr)
    
    # Train and build the encoder-decoder model
    enc_model, dec_model, n_epochs = train_and_build_model(num_characters, max_word_length, encoder_input_data, decoder_input_data, decoder_target_data, val_data)
        
    # Evaluate model performance on training and validation set
    print("Testing real verb performance")
    train_acc, val_acc, train_reg_err, train_irreg_err, val_reg_err, val_irreg_err, val_irreg_COR = compute_acc(label, train_eid, decode_sequence, enc_model, dec_model, char_index, train_target_verbs, val_target_verbs, pairs, val_eid, max_word_length)
    
    # Do Wug Test
    print("Wug Test / nonce verb predictions")
    
    # For each run compute Wug Test production probabilities
    top1_forms, wug_result, beam_per_wug =[],[],[]
    wug_result.append('model '+str(n+1))
    for wug_reg, wug_irreg1, wug_irreg2, wug_other in zip(wugs_reg, wugs_irreg1, wugs_irreg2, wugs_other):
        count = [[wug_reg,0.0],[wug_irreg1,0.0],[wug_irreg2,0.0],[wug_other,0.0]]
        wug_result.append(count)
    
    # Save for each nonce verb in Wug Test results
    for i,w in enumerate(wug_input_data): 
        # Beam search per wug
        beam = beam_search(wug_input_data[i:i+1], enc_model, dec_model, beam_width=beam_width, max_len=max_word_length, char_index=char_index) 
        total_sum = 0

        #For each of the 12 beam predictions decode the outputs to characters
        for ib,b in enumerate(beam):
            total_sum+=b['prob']
            sequence = b['total_seq'].numpy().tolist()[0]
            decoded_verb = ""
            for char in sequence:
                decoded_verb += reverse_char_index[char]
            beam_per_wug.append([i, ib, decoded_verb.strip(), b['prob']])
            
            # Sort the beam predictions and probabilities into categories: regular/irregular1/irregular2/other
            if decoded_verb.strip() == wugs_reg[i]:
                wug_result[i+1][0][1] += b['prob'] # E.g. if beam prediction = regular inflection, reg += current beam probability 
                if ib==0: # Also save top 1 from beam
                    top1_forms.append([decoded_verb.strip(),'reg',b['prob']]) 
            elif decoded_verb.strip() == wugs_irreg1[i]:
                wug_result[i+1][1][1] += b['prob']
                if ib==0:
                    top1_forms.append([decoded_verb.strip(),'irreg1',b['prob']])
            elif decoded_verb.strip() == wugs_irreg2[i]:
                wug_result[i+1][2][1] += b['prob']
                if ib==0:
                    top1_forms.append([decoded_verb.strip(),'irreg2',b['prob']])
            else:
                wug_result[i+1][3][1] += b['prob']
                if ib==0:
                    top1_forms.append([decoded_verb.strip(),'other', b['prob']])

    # Normalising beam probabilities per wug
    for n,w in enumerate(wug_result[1:]):
        summ=w[0][1]+w[1][1]+w[2][1]+w[3][1]
        w[0][1]/=summ
        w[1][1]/=summ
        w[2][1]/=summ
        w[3][1]/=summ

    # Compute correlation coefficients based on wug predictions of current run
    all_correlations = print_correlations(wug_result[1:])

    # Saving the results of the current model
    individual_result = [model_nr, fold, n_epochs, train_acc, val_acc, all_correlations, wug_result[1:], 
                         top1_forms, beam_per_wug, train_reg_err, train_irreg_err,
                         val_reg_err, val_irreg_err, val_irreg_COR]
    
    individual_results.append(individual_result) 

    # Update fold after 5 runs and re-load data, depending on the experiment 
    if model_nr==5 and not exp1: 
        fold = 'second20'
        pairs, written_pairs, freq, label, split = all_data(square_root_normalization, SEED, val_split, fold, excl_extra_verbs, remove_doubles)
        encoder_input_data, train_eid, train_target_verbs, decoder_input_data, decoder_target_data, val_eid, val_target_verbs, char_index, max_word_length, num_characters, val_data = create_model_data(split, pairs, freq, use_token_freq)
    elif model_nr==10:
        fold = 'third20'
        pairs, written_pairs, freq, label, split = all_data(square_root_normalization, SEED, val_split, fold, excl_extra_verbs, remove_doubles)
        encoder_input_data, train_eid, train_target_verbs, decoder_input_data, decoder_target_data, val_eid, val_target_verbs, char_index, max_word_length, num_characters, val_data = create_model_data(split, pairs, freq, use_token_freq)
    elif model_nr==15:
        fold = 'fourth20'
        pairs, written_pairs, freq, label, split = all_data(square_root_normalization, SEED, val_split, fold, excl_extra_verbs, remove_doubles)
        encoder_input_data, train_eid, train_target_verbs, decoder_input_data, decoder_target_data, val_eid, val_target_verbs, char_index, max_word_length, num_characters, val_data = create_model_data(split, pairs, freq, use_token_freq)
    elif model_nr==20 and exp_1_2:
        fold='DEFAULT'
        pairs, written_pairs, freq, label, split = all_data(square_root_normalization, SEED, val_split, fold, excl_extra_verbs, remove_doubles)
        encoder_input_data, train_eid, train_target_verbs, decoder_input_data, decoder_target_data, val_eid, val_target_verbs, char_index, max_word_length, num_characters, val_data = create_model_data(split, pairs, freq, use_token_freq)


    
    #clear session for memory efficiency
    tf.keras.backend.clear_session()


In [ ]:
df=pd.DataFrame(individual_results)
df.columns = df.iloc[0]
df = df[1:]
df = pd.DataFrame(individual_results)
df.to_excel("sgt.xlsx",sheet_name='Sheet_name_1')

In [ ]:
df